# Multi-Label SVM

In [ ]:
import itertools

import numpy as np
from scipy import sparse
from pystruct.datasets import load_scene

scene = load_scene()
X_train, X_test = scene['X_train'], scene['X_test']
y_train, y_test = scene['y_train'], scene['y_test']

n_labels = y_train.shape[1]

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
from pystruct.learners import OneSlackSSVM
from pystruct.models import MultiLabelClf

independent_model = MultiLabelClf(inference_method='unary')
independent_ssvm = OneSlackSSVM(independent_model, C=.1, tol=0.01)
independent_ssvm.fit(X_train, y_train)

In [ ]:
independent_ssvm.score(X_test, y_test)

In [ ]:
independent_ssvm.w.shape

In [ ]:
X_train.shape[1] * y_train.shape[1]

In [ ]:
full_edges = np.vstack([x for x in itertools.combinations(range(n_labels), 2)])
# edges are the same for each sample, so can put them into the model
full_model = MultiLabelClf(edges=full_edges, inference_method='qpbo')
full_ssvm = OneSlackSSVM(full_model, inference_cache=50, C=.1, tol=0.01)
full_ssvm.fit(X_train, y_train)

In [ ]:
full_ssvm.score(X_test, y_test)

In [ ]:
full_ssvm.w.shape

In [ ]:
full_ssvm.w.shape[0] - independent_ssvm.w.shape[0]

In [ ]:
full_edges.shape  # 15 edges, each with 4 parameters (2x2 matrix of compatibilities)

In [ ]:
from sklearn.metrics import mutual_info_score
from scipy.sparse.csgraph import minimum_spanning_tree

def chow_liu_tree(y_):
    # compute mutual information using sklearn
    n_labels = y_.shape[1]
    mi = np.zeros((n_labels, n_labels))
    for i in range(n_labels):
        for j in range(n_labels):
            mi[i, j] = mutual_info_score(y_[:, i], y_[:, j])
    mst = minimum_spanning_tree(sparse.csr_matrix(-mi))
    edges = np.vstack(mst.nonzero()).T
    edges.sort(axis=1)
    return edges

# Exercise
1) Run a model using edges produced by the Chow-Liu tree. Use ``inference='max-product'`` as exact inference can be run in a tree. How many parameters do you expect the model to have?
2) Compare the training and test accuracy of the three models, as well as the runtime.